Okay, to start off I need to import pandas, the "import pandas as pd lets me us pd as shorthand when I call on pandas

In [3]:
import pandas as pd


Next I add in my datasets. I'll start with just 1 set, and print the begining of the dataset to make sure its reading correctly

In [4]:
calendar = pd.read_csv('Datasets\calendar.csv')
print(calendar.shape)
calendar.head(5)

Now the other datasets

In [7]:
normal_sales = pd.read_csv('Datasets/MOCK_BAR_DATA_NO_POPUP.csv')
halloween = pd.read_csv('Datasets/MOCK_BAR_DATA_HALLOWEEN.csv')
holiday = pd.read_csv('Datasets/MOCK_BAR_DATA_SSPOPUP.csv')

I've seperated the mock sales into 3 sets for the sake of making the mock data easier to read. The holiday popups are generally much different than the rest of the year so I created different guidelines for the mock data for those. It also provides the opportunity to start with halloween event to show the concept on a smaller scale.

In [9]:
print (halloween.head (7))

         date  holiday drink 1  holiday drink 2  holiday drink 3  \
0  10/24/2023                2                6                4   
1  10/25/2023                2               10               15   
2  10/26/2023               10                5               11   
3  10/27/2023                8               10                2   
4  10/28/2023               18               14               12   
5  10/29/2023                6                2                9   
6  10/31/2023               13               14                3   

   holiday drink 4  holiday drink 5  holiday shot  holiday bowl  draft beer  \
0                7                2             5           4.0         5.0   
1                3                5             4           NaN         NaN   
2                3                9             2           NaN         7.0   
3                6                3             1           3.0         7.0   
4               15               14            14           